In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0


In [5]:

class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    
    def call(self, input_tensor, training=False):
        x=self.conv(input_tensor)
        x=self.bn(x, training=training)
        x=tf.nn.relu(x)
        return x

model=keras.Sequential([CNNBlock(32),
                        CNNBlock(64),
                        CNNBlock(128),
                        layers.Flatten(),
                        layers.Dense(10)
])

class ResBlock():
    def __init__(self,channels):
        super(ResBlock, self).__init__()
        self.cnn1=CNNBlock(channels[0])
        self.cnn1=CNNBlock(channels[1])
        self.cnn1=CNNBlock(channels[2])
        self.pooling=layers.MaxPooling2D()
        self.identity_mapping= layers.Conv2D(channels[1],1,padding='same')
    
    def call(self, input_tensor, training=False):
        x=self.cnn1(input_tensor, training=training)
        x=self.cnn1(x, training=training)
        x=self.cnn3(x+self.identity_mapping(input_tensor), training=training)
        return self.pooling(x)
    
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1=ResBlock([32,32,64])
        self.block2=ResBlock([128,128,256])
        self.block3=ResBlock([128,256,512])
    

In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'],)
model.fit(x_train,y_train, batch_size=64,epochs=3,verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/3
938/938 - 31s - loss: 0.6177 - accuracy: 0.9430
Epoch 2/3
938/938 - 25s - loss: 0.0996 - accuracy: 0.9808
Epoch 3/3
938/938 - 25s - loss: 0.0389 - accuracy: 0.9884
157/157 - 2s - loss: 0.0557 - accuracy: 0.9853


[0.05566613748669624, 0.9853000044822693]

## SubClassing 

In [7]:
#So first we create the CNNBlock using conv, batch, relu, cuz 
#we want to reuse it 

#Then we use the ResBlock that uses these blocks multiple times 
#and also a pooling layer multiple times
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x 


model = keras.Sequential(
    [CNNBlock(32), CNNBlock(64), CNNBlock(128), layers.Flatten(), layers.Dense(10),]
)


In [8]:
#By now we may have noted,e every layer has a init and a 
#call method
#Further we'll see theres also a model method. We can assume this 
#is how the structure goes for subclassing and its predefined

class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,)
        x = self.pooling(x)
        return x


In [9]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

    def model(self):
        x = keras.Input(shape=(28, 28, 1))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [10]:
model = ResNet_Like().model()
base_input = model.layers[0].input
base_output = model.layers[2].output
output = layers.Dense(10)(layers.Flatten()(base_output))
model = keras.Model(base_input, output)

In [11]:

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [12]:
model.fit(x_train, y_train, batch_size=64, epochs=1, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)
model.save("pretrained")

938/938 - 37s - loss: 0.1054 - accuracy: 0.9686
157/157 - 2s - loss: 0.0305 - accuracy: 0.9899


INFO:tensorflow:Assets written to: pretrained\assets


INFO:tensorflow:Assets written to: pretrained\assets
